# Modelos de Regresión y Clasificación II
Actividad Lección 4 || Fundamentos de IA y Machine Learning

Objetivos:
* Aplicar conceptos teóricos vistos en clase

Datos del alumno:
* Víctor Luque Martín
* Máster Avanzado en Programación en Python para Hacking, BigData y Machine Learning

Fecha: 21/10/2022

# Tabla de contenidos
1. [Problema I](#pi)
    1. [Preparación previa](#pi-pp)
        1. [Función activación](#pi-pp-fa)
        2. [Función sigmoide](#pi-pp-fs)
        3. [Función a optimizar o predicción](#pi-pp-fo)
    2. [Resultados](#pi-res)
    3. [Métricas](#pi-met)
        1. [Matriz de confusión](#pi-met-mc)
        2. [CCR y Kappa](#pi-met-ccr-k)
        3. [Matrices de confusión para cada clase](#pi-met-mcc)
        4. [Métricas para cada matriz de confusión de cada clase](#pi-met-mcc-stat)
    4. [Conclusiones](#pi-con)

In [1]:
import pandas as pd
import math
from IPython.display import display

# Problema I <a class="anchor" id="pi"></a>
Dada la siguiente red neuronal entrenada:

![](img/red_neuronal_entrenada.png)

In [2]:
l4p1_capa_entrada = pd.DataFrame({
    "bias": [6.46, 19.64],
    "X1": [-1.6, -1.35],
    "X2": [-3.43, -7.52],
})
l4p1_capa_entrada

,bias,X1,X2
0,6.46,-1.60,-3.43
1,19.64,-1.35,-7.52


In [3]:
l4p1_capa_oculta = pd.DataFrame({
    "bias": [-5.47, -3.76, 3.96],
    "B1": [7.61, -8.21, -3.81],
    "B2": [1.9, 7.87, -8.46]
})
l4p1_capa_oculta

,bias,B1,B2
0,-5.47,7.61,1.90
1,-3.76,-8.21,7.87
2,3.96,-3.81,-8.46


**Se pide hallar:**
- Hallar las predicciones del modelo para el siguiente conjunto de test.
- Evaluar el rendimiento del clasificador en dicho conjunto.

In [4]:
l4p1_test_df = pd.read_csv('l4p1_test.csv')
l4p1_test_df

,x1,x2,clase
0,1.5,0.2,1
1,1.4,0.3,1
2,1.6,0.4,1
3,1.1,0.1,2
4,4.3,1.3,1
5,3.0,1.1,2
6,4.9,2.0,2
7,6.1,1.9,2
8,4.4,1.2,3
9,5.9,2.1,1


## Preparación previa <a class="anchor" id="pi-pp"></a>
### Función activación <a class="anchor" id="pi-pp-fa"></a>
Se utilizará una red neuronal sigmoide (SUNN) cuya función de activación consiste en un modelo aditivo. La función de activación se define como:

$$B_j(x, w_j) = h (w_{0,j} + \sum_{i=1}^{n} w_{i,j} x_i)$$

In [5]:
def f_act_add(row, i):
    # w_oj
    z = l4p1_capa_entrada.loc[i, "bias"]
    i = l4p1_capa_entrada.iloc[i, 1:]
    ec = f"{z}"
    for idx in range(len(i)):
        # sumatorio
        z += i[idx] * row[idx]
        ec += " + " + str(i[idx]) + " * " + str(row[idx])
    return z

### Función sigmoide <a class="anchor" id="pi-pp-fs"></a>
Aplicada a una red neuronal sigmoide (SUNN), la función de activación (función sigmoide) se define como:

$$B_j(x, w_j) = \frac{1}{1 + e^{- (w_{0,j} + \sum_{i=1}^{n} w_{i,j} x_i)}}$$

In [6]:
def f_sigmoide(z):
    return 1 / (1 + math.exp(-(z)))

### Función a optimizar o predicción <a class="anchor" id="pi-pp-fo"></a>
Para predecir los resultados, se debe emplear la siguiente expresión:

$$\hat{y}(x, \theta) = h(\beta_0 + \sum_{j=1}^{M} \beta_j B_j (x, w_j))$$

Aplicando la función sigmoide se vería de la siguiente forma:

$$\hat{y}(x, \theta) = \frac{1}{1 + e^{- (\beta_0 + \sum_{j=1}^{M} \beta_j B_j (x, w_j))}}$$

In [7]:
def funcion_optimizar(row, j):
    # beta_0
    z = l4p1_capa_oculta.loc[j, "bias"]
    # selecciono la el valor de B1, B2, B3
    i = l4p1_capa_oculta.iloc[j, 1:]
    ec = str(z)
    # Calculo Bj
    #print(f"- B{j+1}")
    ec = f"{z}"
    for idx in range(len(i)):
        # sumatorio
        z += i[idx] * row[idx]
        ec += " + " + str(i[idx]) + " * " + str(row[idx])
    return f_sigmoide(z)

## Resultados <a class="anchor" id="pi-res"></a>

In [8]:
df_capa_oculta = pd.DataFrame({
    f"B{j+1}": [f_sigmoide(f_act_add(row, j)) for _, row in l4p1_test_df.iterrows()] 
    for j in range(2)
})
df_capa_salida = pd.DataFrame({
    f"C{i+1}": [funcion_optimizar(row, i) for _, row in df_capa_oculta.iterrows()] 
    for i in range(3)
})
# Movemos la clase real al final del df
l4p1_test_df[["B1", "B2"]] = df_capa_oculta[["B1", "B2"]]
l4p1_test_df[["C1", "C2", "C3"]] = df_capa_salida[["C1", "C2", "C3"]]
l4p1_test_df["Predicha"] = l4p1_test_df[["C1", "C2", "C3"]]\
    .idxmax(axis=1).str[-1].astype(int)
l4p1_test_df["Real"] = l4p1_test_df["clase"]
l4p1_test_df.drop(columns=['clase'], inplace=True)
l4p1_test_df

,x1,x2,B1,B2,C1,C2,C3,Predicha,Real
0,1.5,0.2,0.966882,1.000000,0.977860,0.021288,0.000279,1,1
1,1.4,0.3,0.960494,1.000000,0.976783,0.022408,0.000286,1,1
2,1.6,0.4,0.926082,0.999999,0.970040,0.029508,0.000326,1,1
3,1.1,0.1,0.987345,1.000000,0.980992,0.018055,0.000258,1,2
4,4.3,1.3,0.007547,0.983027,0.028064,0.980439,0.012308,2,1
5,3.0,1.1,0.107840,0.999337,0.060053,0.961559,0.007353,2,2
6,4.9,2.0,0.000264,0.117637,0.005249,0.055391,0.950911,3,2
7,6.1,1.9,0.000055,0.053000,0.004638,0.034114,0.971011,3,2
8,4.4,1.2,0.009049,0.990770,0.028786,0.981350,0.011471,2,3
9,5.9,2.1,0.000038,0.016032,0.004324,0.025727,0.978631,3,1


## Métricas <a class="anchor" id="pi-met"></a>
Se reutiliza el objeto de MetricasClasificación utilizado en anteriores lecciones para calcular las métricas del modelo.

In [9]:
class MetricasClasificacion:
    def __init__(self, y, y_pred) -> None:
        self.y = y
        self.y_pred = y_pred
        self.n = len(y)
        self.matriz_confusion = pd.crosstab(self.y,
                                            self.y_pred, 
                                            rownames=["Real"], 
                                            colnames=["Predicha"])

    def crear_sub_matriz(self, c):
        matriz = self.matriz_confusion.copy()
        matriz["Negativo"] = matriz.sum(axis=1) - matriz[c]
        matriz["Positivo"] = matriz[c]
        matriz = matriz[["Positivo", "Negativo"]].rename(index={c: "Positivo"})
        matriz.loc["Negativo"] = matriz.sum(axis=0) - matriz.loc["Positivo"]
        matriz = matriz.drop(matriz.index.difference(["Positivo", "Negativo"]))
        return matriz

    def ccr(self):
        confusion = self.matriz_confusion.copy()
        columnas = confusion.columns
        filas = confusion.index
        suma_matriz = confusion.sum().sum()
        numerador = 0
        for c, f in zip(columnas, filas):
            if columnas.get_loc(c) == filas.get_loc(f):
                numerador += confusion[c][f]
        return numerador / suma_matriz
            
    def tpr(self, c = None):
        try:
            if c is not None:
                confusion = self.crear_sub_matriz(c)
            else:
                confusion = self.matriz_confusion.copy()
            return confusion["Positivo"]["Positivo"] / \
                (confusion["Positivo"]["Positivo"] + confusion["Negativo"]["Positivo"])
        except Exception:
            return 0

    def fpr(self, c = None):
        try:    
            if c is not None:
                confusion = self.crear_sub_matriz(c)
            else:
                confusion = self.matriz_confusion.copy()
            return confusion["Positivo"]["Negativo"] / \
                (confusion["Negativo"]["Negativo"] + confusion["Positivo"]["Negativo"])
        except Exception:
            return 0

    def tnr(self, c = None):
        try:
            if c is not None:
                confusion = self.crear_sub_matriz(c)
            else:
                confusion = self.matriz_confusion.copy()
            return confusion["Negativo"]["Negativo"] / \
                (confusion["Negativo"]["Negativo"] + confusion["Positivo"]["Negativo"])
        except Exception:
            return 0

    def ppv(self, c = None):
        try:
            if c is not None:
                confusion = self.crear_sub_matriz(c)
            else:
                confusion = self.matriz_confusion.copy()
            return confusion["Positivo"]["Positivo"] / \
                (confusion["Positivo"]["Positivo"] + confusion["Positivo"]["Negativo"])
        except Exception:
            return 0

    def f1(self, c = None):
        try:
            return 2 * (self.ppv(c) * self.tpr(c)) / \
                (self.ppv(c) + self.tpr(c))
        except Exception:
            return 0

    def kappa(self):
        confusion = self.matriz_confusion.copy()
        confusion.loc["Total"] = confusion.sum()
        confusion["Total"] = confusion.sum(axis=1)
        total_col = confusion["Total"]
        total_row = confusion.loc["Total"]
        total = confusion.loc["Total", "Total"]
        total_row = total_row.drop("Total")
        total_col = total_col.drop("Total")
        pe = 0
        for c, f in zip(total_col.index, total_row.index):
            pe += (total_col[c] * total_row[f])/(total*total)
        k = (self.ccr() - pe) / (1 - pe)
        return k

### Matriz de confusion <a class="anchor" id="pi-met-mc"></a>

In [10]:
l4p1_metricas = MetricasClasificacion(l4p1_test_df["Real"], l4p1_test_df["Predicha"])
l4p1_metricas.matriz_confusion

Predicha,1,2,3
Real,,,
1,3,1,1
2,1,1,2
3,0,2,1


### CCR y Kappa <a class="anchor" id="pi-met-ccr-k"></a>

In [11]:
print(f"CCR: {l4p1_metricas.ccr()}")
print(f"Kappa: {l4p1_metricas.kappa()}")

CCR: 0.4166666666666667
Kappa: 0.12500000000000003


### Matrices de confusion para cada clase <a class="anchor" id="pi-met-mcc"></a>

In [12]:
for i in range(1, 4):
    print("=============================")
    print(f"Matriz Confusión - Clase {i}")
    display(l4p1_metricas.crear_sub_matriz(i))

Matriz Confusión - Clase 1


Predicha,Positivo,Negativo
Real,,
Positivo,3,2
Negativo,1,6


Matriz Confusión - Clase 2


Predicha,Positivo,Negativo
Real,,
Positivo,1,3
Negativo,3,5


Matriz Confusión - Clase 3


Predicha,Positivo,Negativo
Real,,
Positivo,1,2
Negativo,3,6


### Métricas para cada matriz de confusión de cada clase <a class="anchor" id="pi-met-mcc-stat"></a>
- Sensibilidad
- False Positive Rate
- Especificidad
- Precisión
- F1 Score

In [13]:
l3p1_metricas_data = [
    {
        "Sensibilidad": l4p1_metricas.tpr(i), "False Positive Rate": l4p1_metricas.fpr(i),
        "Especificidad": l4p1_metricas.tnr(i), "Precision": l4p1_metricas.ppv(i),
        "F1 Score": l4p1_metricas.f1(i)
    } for i in range(1, 4)
]
l3p1_df_metricas = pd.DataFrame(l3p1_metricas_data, index=["1", "2", "3"])
l3p1_df_metricas.index.name = "Clase"
l3p1_df_metricas.loc["Promedio"] = l3p1_df_metricas.mean()
l3p1_df_metricas = l3p1_df_metricas.style.apply(
    lambda x: ["background: yellow" if x.name == "Promedio" else "" for i in x], axis=1)
l3p1_df_metricas

,Sensibilidad,False Positive Rate,Especificidad,Precision,F1 Score
Clase,,,,,
1,0.600000,0.142857,0.857143,0.750000,0.666667
2,0.250000,0.375000,0.625000,0.250000,0.250000
3,0.333333,0.333333,0.666667,0.250000,0.285714
Promedio,0.394444,0.283730,0.716270,0.416667,0.400794


## Conclusiones <a class="anchor" id="pi-con"></a>
- El modelo tiene un CCR bajo (0,416) y un Kappa bajo (0,125). Esto indica que el modelo no es bueno para predecir los resultados.
- La sensibilidad y la especificidad de la clase 1 es superior al del resto de clases.
- La tasa de falsos positivos de la clase 1 es inferior al del resto de clases.
- La precisión del modelo 1 es del 75% mientras que tanto de la clase 2 como de la clase 3 es d eun 25%.
- El F1 Score de la clase 1 es superior al del resto de clases.
- El modelo no es bueno para predecir los resultados, siendo la clase 1 la que mejor predicción tiene.